In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.discrete.discrete_model as smdiscrete
import seaborn as sns

In [2]:
# mode.use_inf_as_na, boolean(nan都变为inf无穷大)
# True means treat None, NaN, INF, -INF as NA (old way), 
# False means None and NaN are null, but INF, -INF are not NA (new way). [default: False] [currently: False]
pd.set_option('use_inf_as_na', True)

meta_df = pd.read_csv("stockmetadata.csv")
fdata_df = pd.read_csv("corpfund.csv")

# We only keep "as reported data"(ARQ) to avoid forward bias due to corporate restatements
fdata_df = fdata_df[fdata_df['dimension']=='ARQ']

In [3]:
fdata_df['datekey'] = pd.to_datetime(fdata_df['datekey'])
# we merge both dataframes based on ticker
df_left = pd.merge(fdata_df, meta_df, on='ticker', how='left')

# we set index of the dataframe to be data- date
df_left = df_left.set_index('datekey')
df_left.head()

,ticker,dimension,calendardate,reportperiod,lastupdated_x,accoci,assets,assetsavg,assetsc,assetsnc,...,currency,location,lastupdated_y,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
datekey,,,,,,,,,,,,,,,,,,,,,
2010-06-07,A,ARQ,2010-03-31,2010-04-30,2020-09-01,-239000000.0,7.767000e+09,NaN,5.712000e+09,2.055000e+09,...,USD,California; U.S.A,2020-09-01,2014-09-26,1999-11-18,2020-10-09,1997-06-30,2020-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
2010-09-07,A,ARQ,2010-06-30,2010-07-31,2020-09-01,-225000000.0,9.100000e+09,NaN,5.735000e+09,3.365000e+09,...,USD,California; U.S.A,2020-09-01,2014-09-26,1999-11-18,2020-10-09,1997-06-30,2020-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
2010-12-20,A,ARQ,2010-09-30,2010-10-31,2020-09-01,-88000000.0,9.696000e+09,NaN,6.169000e+09,3.527000e+09,...,USD,California; U.S.A,2020-09-01,2014-09-26,1999-11-18,2020-10-09,1997-06-30,2020-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
2011-03-09,A,ARQ,2010-12-31,2011-01-31,2020-09-01,-63000000.0,8.044000e+09,NaN,4.598000e+09,3.446000e+09,...,USD,California; U.S.A,2020-09-01,2014-09-26,1999-11-18,2020-10-09,1997-06-30,2020-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
2011-06-07,A,ARQ,2011-03-31,2011-04-30,2020-09-01,278000000.0,8.649000e+09,NaN,5.096000e+09,3.553000e+09,...,USD,California; U.S.A,2020-09-01,2014-09-26,1999-11-18,2020-10-09,1997-06-30,2020-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com


> We want to estimate relationship between `E/P ratio` and `operating profit margin` for each of these data point.

> However, we note different industries (`"sicsector"`) may have completely different `operating margins` on average, as well as different relationships between `operating margin` and `E/P ratio`.

> One solution is to introduce an indicator variable (“`dummy variable`”) for each industry.

> This allows each industry to effectively have its own y-intercept.

> E/P ratio = $\frac{eps}{price}$

> E/P ratio = $\frac{eps}{(price + k)}$

### OLS without fixed effects:

$$ Y_i =α+βX_i +ε $$

### OLS with fixed effects:

$$ Y_i =α+β_1X_i +β_2*I_1 +β_3*I_2 +β_4*I_3 +...β_{12}*I_{12} +ε $$

#### Where

<center>$I_1 = 1$ if firm is in sector 1, 0 otherwise </center>

<center>$I_2 = 1$ if firm is in sector 2, 0 otherwise </center>

    ...

<center>$I_{12} = 1$ if firm is in sector 12, 0 otherwise </center>

> `industrydummies` is consist of $I_1 - I_9$.

In [4]:
# we generate 1 dummy variable for each sector
# pd.get_dummies: Convert categorical variable into dummy/indicator variables
industrydummies = pd.get_dummies(df_left['sicsector'])
industrydummies

,Agriculture Forestry And Fishing,Construction,Finance Insurance And Real Estate,Manufacturing,Mining,Retail Trade,Services,Transportation Communications Electric Gas And Sanitary Service,Wholesale Trade
datekey,,,,,,,,,
2010-06-07,0,0,0,1,0,0,0,0,0
2010-09-07,0,0,0,1,0,0,0,0,0
2010-12-20,0,0,0,1,0,0,0,0,0
2011-03-09,0,0,0,1,0,0,0,0,0
2011-06-07,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2020-10-05,0,0,0,1,0,0,0,0,0
2020-10-08,0,0,0,1,0,0,0,0,0
2020-10-08,0,0,0,1,0,0,0,0,0


In [5]:
industrydummies.sum()         #purely for exploring the data, has no other purpose

Agriculture Forestry And Fishing                                     563
Construction                                                        1900
Finance Insurance And Real Estate                                  36533
Manufacturing                                                      61561
Mining                                                              7558
Retail Trade                                                        7638
Services                                                           23859
Transportation Communications Electric Gas And Sanitary Service    10911
Wholesale Trade                                                     3988
dtype: int64

In [6]:
industrydummies.describe()      #purely for exploring the data, has no other purpose

,Agriculture Forestry And Fishing,Construction,Finance Insurance And Real Estate,Manufacturing,Mining,Retail Trade,Services,Transportation Communications Electric Gas And Sanitary Service,Wholesale Trade
count,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000,207494.000000
mean,0.002713,0.009157,0.176068,0.296688,0.036425,0.036811,0.114986,0.052585,0.019220
std,0.052019,0.095253,0.380879,0.456799,0.187346,0.188297,0.319006,0.223203,0.137297
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# we merge the dummy variable dataframe into the main dataframe
data_w_dummies = pd.concat([df_left,industrydummies], axis=1)

# we drop 1 dummy variable, which will be the control group(控制组)
data_w_dummies.drop(['Wholesale Trade'], inplace=True, axis=1)

# we generate our dependent variable (E/P ratio), eps (earnings per share)
data_w_dummies['epratio'] = data_w_dummies['eps']/data_w_dummies['price']

# we generate our independent variable, which is operating margin, opinc (operating income)
data_w_dummies['operatingmargin'] = data_w_dummies['opinc'] / data_w_dummies['revenue']

In [8]:
#initial analysis
result = sm.OLS(data_w_dummies['epratio'], 
                sm.add_constant(data_w_dummies[['operatingmargin']]), 
                missing='drop'
               ).fit()
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                epratio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                 6.662e-06
Date:                Fri, 05 Nov 2021   Prob (F-statistic):              0.998
Time:                        23:49:14   Log-Likelihood:            -1.0536e+06
No. Observations:              187087   AIC:                         2.107e+06
Df Residuals:                  187085   BIC:                         2.107e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0877      0.156      0.561      0.574      -0.218       0.394
operatingmargin  8.972e-08   3.48e-05      0.003      0.998    -6.8e-05    6.82e-05
==============================================================================
Omnibus:                   940458.507   Durbin-Watson:                   2.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   34055474686020.352
Skew:                         219.936   Prob(JB):                         0.00
Kurtosis:                   66097.857   Cond. No.                     4.49e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
result = sm.OLS(data_w_dummies['epratio'],
                sm.add_constant(data_w_dummies[['operatingmargin',
                                                'Agriculture Forestry And Fishing',
                                                'Construction',
                                                'Finance Insurance And Real Estate',
                                                'Manufacturing',
                                                'Mining',
                                                'Retail Trade',
                                                'Services',
                                                'Transportation Communications Electric Gas And Sanitary Service']]),
                missing='drop'
               ).fit()
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                epratio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1101
Date:                Fri, 05 Nov 2021   Prob (F-statistic):              0.999
Time:                        23:49:31   Log-Likelihood:            -1.0536e+06
No. Observations:              187087   AIC:                         2.107e+06
Df Residuals:                  187077   BIC:                         2.107e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.3101      0.296      1.047      0.295      -0.270       0.890
operatingmargin                                                  9.048e-08   3.48e-05      0.003      0.998    -6.8e-05    6.82e-05
Agriculture Forestry And Fishing                                   -0.3074      3.019     -0.102      0.919      -6.226       5.611
Construction                                                       -0.3198      1.597     -0.200      0.841      -3.450       2.811
Finance Insurance And Real Estate                                  -0.3499      0.470     -0.744      0.457      -1.271       0.572
Manufacturing                                                      -0.2588      0.417     -0.621      0.534      -1.075       0.558
Mining                                                             -0.5657      0.920     -0.615      0.539      -2.368       1.237
Retail Trade                                                       -0.3552      0.843     -0.422      0.673      -2.007       1.296
Services                                                           -0.2146      0.546     -0.393      0.694      -1.284       0.855
Transportation Communications Electric Gas And Sanitary Service    -0.4327      0.726     -0.596      0.551      -1.855       0.990
==============================================================================
Omnibus:                   940450.115   Durbin-Watson:                   2.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   34053234843071.676
Skew:                         219.929   Prob(JB):                         0.00
Kurtosis:                   66095.683   Cond. No.                     8.69e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.69e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### We note t-statistic for estimated coefficient on ‘operatingmargin’ is very low

> This might be due to the effect of outliners overly influencing estimation.

> Two ways to explore the effect of outliers are:

> • 1. Winsorize data (albeit at expense of deleting observations arbitrarily (“datamining”))

> • 2. Apply `logarithm` to both dependent and independent variables. This will naturally reduce impact of outliers

In [10]:
# Generate log versions of independent variable
data_w_dummies['lnoperatingmargin'] = np.log(data_w_dummies['operatingmargin'])
result = sm.OLS(data_w_dummies['epratio'], 
                sm.add_constant(data_w_dummies[['lnoperatingmargin']]), 
                missing='drop'
               ).fit()
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                epratio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1565
Date:                Sat, 06 Nov 2021   Prob (F-statistic):              0.692
Time:                        00:18:52   Log-Likelihood:            -7.2573e+05
No. Observations:              130525   AIC:                         1.451e+06
Df Residuals:                  130523   BIC:                         1.451e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2760      0.363      0.760      0.447      -0.436       0.988
lnoperatingmargin     0.0601      0.152      0.396      0.692      -0.238       0.358
==============================================================================
Omnibus:                   708882.580   Durbin-Watson:                   2.592
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   58382860353969.094
Skew:                         290.730   Prob(JB):                         0.00
Kurtosis:                  103611.316   Cond. No.                         5.69
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# Generate log versions of both dependent and independent variables
data_w_dummies['lnepratio'] = np.log(data_w_dummies['epratio'])
result = sm.OLS(data_w_dummies['lnepratio'],
                sm.add_constant(data_w_dummies[['lnoperatingmargin']]),
                missing='drop'
               ).fit()
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     8846.
Date:                Sat, 06 Nov 2021   Prob (F-statistic):               0.00
Time:                        00:19:17   Log-Likelihood:            -1.5681e+05
No. Observations:              115861   AIC:                         3.136e+05
Df Residuals:                  115859   BIC:                         3.136e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -3.7721      0.006   -602.657      0.000      -3.784      -3.760
lnoperatingmargin     0.2638      0.003     94.054      0.000       0.258       0.269
==============================================================================
Omnibus:                    18365.631   Durbin-Watson:                   1.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           158342.851
Skew:                           0.510   Prob(JB):                         0.00
Kurtosis:                       8.635   Cond. No.                         5.93
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# with dummy variables
result = sm.OLS(data_w_dummies['lnepratio'],
                sm.add_constant(data_w_dummies[['lnoperatingmargin',
                                                'Agriculture Forestry And Fishing',
                                                'Construction',
                                                'Finance Insurance And Real Estate',
                                                'Manufacturing',
                                                'Mining',
                                                'Retail Trade',
                                                'Services',
                                                'Transportation Communications Electric Gas And Sanitary Service']]
                               ),
                missing='drop'
               ).fit()
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     1188.
Date:                Sat, 06 Nov 2021   Prob (F-statistic):               0.00
Time:                        00:20:12   Log-Likelihood:            -1.5595e+05
No. Observations:              115861   AIC:                         3.119e+05
Df Residuals:                  115851   BIC:                         3.120e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                              -3.6332      0.009   -419.187      0.000      -3.650      -3.616
lnoperatingmargin                                                   0.2969      0.003     96.466      0.000       0.291       0.303
Agriculture Forestry And Fishing                                    0.6587      0.058     11.417      0.000       0.546       0.772
Construction                                                        0.3404      0.025     13.352      0.000       0.290       0.390
Finance Insurance And Real Estate                                  -0.1784      0.008    -21.688      0.000      -0.195      -0.162
Manufacturing                                                      -0.0182      0.008     -2.360      0.018      -0.033      -0.003
Mining                                                             -0.0450      0.018     -2.489      0.013      -0.080      -0.010
Retail Trade                                                        0.1256      0.014      9.187      0.000       0.099       0.152
Services                                                           -0.2630      0.010    -26.005      0.000      -0.283      -0.243
Transportation Communications Electric Gas And Sanitary Service    -0.1285      0.012    -10.640      0.000      -0.152      -0.105
==============================================================================
Omnibus:                    18757.262   Durbin-Watson:                   1.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           159948.347
Skew:                           0.533   Prob(JB):                         0.00
Kurtosis:                       8.657   Cond. No.                         51.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### We note t-statistic is significantly improved, but could this be a result of intra-cluster correlations?

> Observations within same industry are likely to be highly correlated versus observations in different industries

> • This violates OLS assumption of iid datapoints

> • Estimates may therefore greatly overstate efficiency (inflated t-statistics, artificially low p- values, downward biases standard errors)

> • We can adjust the standard error estimates by `clustering errors` within each industry

> • Background: See for e.g. https://economics.mit.edu/files/13927 for more information

In [13]:
# clustering standard variables
# We drop all observations with missing data to avoid a bug in python
data_w_dummies.dropna(subset = ['lnepratio', 'lnoperatingmargin'], inplace=True)
# because of a bug in python where fillna is not working perfectly

# note that we can cannot cluster by str variables in python
# hence using siccode(部门编号) instead of sicsector.
# Run OLS estimation on log variables with FE, and standard errors clustered by sic_code.
result = sm.OLS(data_w_dummies['lnepratio'],
                sm.add_constant(data_w_dummies[['lnoperatingmargin',
                                                'Agriculture Forestry And Fishing',
                                                'Construction',
                                                'Finance Insurance And Real Estate',
                                                'Manufacturing',
                                                'Mining',
                                                'Retail Trade',
                                                'Services',
                                                'Transportation Communications Electric Gas And Sanitary Service']]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     44.65
Date:                Sat, 06 Nov 2021   Prob (F-statistic):           2.30e-80
Time:                        00:59:57   Log-Likelihood:            -1.5595e+05
No. Observations:              115861   AIC:                         3.119e+05
Df Residuals:                  115851   BIC:                         3.120e+05
Df Model:                           9                                         
Covariance Type:              cluster                                         
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                              -3.6332      0.034   -107.420      0.000      -3.700      -3.567
lnoperatingmargin                                                   0.2969      0.016     18.678      0.000       0.266       0.328
Agriculture Forestry And Fishing                                    0.6587      0.357      1.847      0.065      -0.040       1.358
Construction                                                        0.3404      0.064      5.292      0.000       0.214       0.467
Finance Insurance And Real Estate                                  -0.1784      0.127     -1.405      0.160      -0.427       0.070
Manufacturing                                                      -0.0182      0.052     -0.351      0.726      -0.120       0.083
Mining                                                             -0.0450      0.048     -0.947      0.344      -0.138       0.048
Retail Trade                                                        0.1256      0.074      1.699      0.089      -0.019       0.270
Services                                                           -0.2630      0.085     -3.105      0.002      -0.429      -0.097
Transportation Communications Electric Gas And Sanitary Service    -0.1285      0.058     -2.215      0.027      -0.242      -0.015
==============================================================================
Omnibus:                    18757.262   Durbin-Watson:                   1.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           159948.347
Skew:                           0.533   Prob(JB):                         0.00
Kurtosis:                       8.657   Cond. No.                         51.5
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

### Generate a 1/0 variable `paydividend` on whether the company pays a dividend

In [17]:
#generate categorical variable for probit/logit analysis
data_w_dummies['paydividend'] = data_w_dummies['dps']>0
data_w_dummies['paydividend']

datekey
2010-06-07    False
2010-09-07    False
2010-12-20    False
2011-03-09    False
2011-06-07    False
              ...  
2020-10-09     True
2020-10-09     True
2020-10-09    False
2016-08-15    False
2016-11-14    False
Name: paydividend, Length: 115861, dtype: bool

In [18]:
data_w_dummies['paydividend'].mean()

0.5197952719206635

In [19]:
data_w_dummies['paydividend'] = data_w_dummies['paydividend'].astype(int)
# formatting the data for estimation models
data_w_dummies['paydividend']

datekey
2010-06-07    0
2010-09-07    0
2010-12-20    0
2011-03-09    0
2011-06-07    0
             ..
2020-10-09    1
2020-10-09    1
2020-10-09    0
2016-08-15    0
2016-11-14    0
Name: paydividend, Length: 115861, dtype: int64

In [20]:
# runs an estimation with 1/0 dependent variable using OLS anyway
result = sm.OLS(data_w_dummies['paydividend'],
                sm.add_constant(data_w_dummies[['lnoperatingmargin',
                                                'Agriculture Forestry And Fishing',
                                                'Construction',
                                                'Finance Insurance And Real Estate',
                                                'Manufacturing',
                                                'Mining','Retail Trade',
                                                'Services',
                                                'Transportation Communications Electric Gas And Sanitary Service']]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            paydividend   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     20.87
Date:                Sat, 06 Nov 2021   Prob (F-statistic):           1.62e-35
Time:                        01:50:38   Log-Likelihood:                -77645.
No. Observations:              115861   AIC:                         1.553e+05
Df Residuals:                  115851   BIC:                         1.554e+05
Df Model:                           9                                         
Covariance Type:              cluster                                         
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.5519      0.019     29.824      0.000       0.516       0.588
lnoperatingmargin                                                   0.0716      0.009      8.018      0.000       0.054       0.089
Agriculture Forestry And Fishing                                    0.0226      0.117      0.192      0.847      -0.208       0.253
Construction                                                       -0.0074      0.057     -0.129      0.897      -0.120       0.105
Finance Insurance And Real Estate                                   0.2932      0.032      9.187      0.000       0.231       0.356
Manufacturing                                                       0.0932      0.028      3.334      0.001       0.038       0.148
Mining                                                              0.0665      0.071      0.933      0.351      -0.073       0.206
Retail Trade                                                        0.1043      0.038      2.754      0.006       0.030       0.178
Services                                                           -0.0502      0.033     -1.519      0.129      -0.115       0.015
Transportation Communications Electric Gas And Sanitary Service     0.2445      0.046      5.288      0.000       0.154       0.335
==============================================================================
Omnibus:                   584262.775   Durbin-Watson:                   0.621
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12110.797
Skew:                          -0.030   Prob(JB):                         0.00
Kurtosis:                       1.417   Cond. No.                         51.5
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

### Estimates a logit model

In [21]:
# use logit instead
result = smdiscrete.Logit(data_w_dummies['paydividend'],
                          sm.add_constant(data_w_dummies[['lnoperatingmargin',
                                                          'Agriculture Forestry And Fishing',
                                                          'Construction',
                                                          'Finance Insurance And Real Estate',
                                                          'Manufacturing',
                                                          'Mining',
                                                          'Retail Trade',
                                                          'Services',
                                                          'Transportation Communications Electric Gas And Sanitary Service']]
                                         ),
                          missing='drop'
                         ).fit(cov_type='cluster',
                               cov_kwds={'groups': data_w_dummies['siccode']}
                              )
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Optimization terminated successfully.
         Current function value: 0.638094
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            paydividend   No. Observations:               115861
Model:                          Logit   Df Residuals:                   115851
Method:                           MLE   Df Model:                            9
Date:                Sat, 06 Nov 2021   Pseudo R-squ.:                 0.07838
Time:                        01:54:32   Log-Likelihood:                -73930.
converged:                       True   LL-Null:                       -80218.
Covariance Type:              cluster   LLR p-value:                     0.000
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.2637      0.088      2.989      0.003       0.091       0.437
lnoperatingmargin                                                   0.3209      0.044      7.249      0.000       0.234       0.408
Agriculture Forestry And Fishing                                    0.0979      0.498      0.196      0.844      -0.879       1.075
Construction                                                       -0.0196      0.258     -0.076      0.940      -0.525       0.486
Finance Insurance And Real Estate                                   1.2749      0.170      7.494      0.000       0.941       1.608
Manufacturing                                                       0.3926      0.116      3.373      0.001       0.164       0.621
Mining                                                              0.2640      0.293      0.900      0.368      -0.311       0.839
Retail Trade                                                        0.4485      0.156      2.868      0.004       0.142       0.755
Services                                                           -0.2165      0.148     -1.460      0.144      -0.507       0.074
Transportation Communications Electric Gas And Sanitary Service     1.0202      0.208      4.903      0.000       0.612       1.428
===================================================================================================================================
"""

## PCA (Principal component analysis)

In [25]:
from sklearn.decomposition import PCA
# AN OPEN SOURCE MACHINE LEARNING LIBRARY THAT HAS THE PCA ALGORITHM IMPLEMENTED.
# THIS LIBRARY IS WIDELY USED IN BOTH RESEARCH ACADEMIA AND ALSO COMMERCIALLY

from sklearn.preprocessing import StandardScaler

In [22]:
data = data_w_dummies  #renaming the variable for easier typing

# GENERATE 28 INDEPENDENT VARIABLES
numerator = ['cashneq', 'debt', 'ebit', 'ebt', 'eps', 'equity', 'fcf', 'gp', 'inventory', 'liabilities', 'payables', 'receivables', 'tangibles', 'workingcapital']
denominator = ['assets', 'revenue']
featureslist = []

# LN(..) OF ECONOMICALLY SIGNIFICANT RATIOS ON THE BALANCE SHEET AND INCOME STATEMENTS
for n in numerator:
    for d in denominator:
        tag = n+'_'+d
        data[tag] = np.log(data[n]/data[d])
        featureslist.append(tag)

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [23]:
featureslist

['cashneq_assets',
 'cashneq_revenue',
 'debt_assets',
 'debt_revenue',
 'ebit_assets',
 'ebit_revenue',
 'ebt_assets',
 'ebt_revenue',
 'eps_assets',
 'eps_revenue',
 'equity_assets',
 'equity_revenue',
 'fcf_assets',
 'fcf_revenue',
 'gp_assets',
 'gp_revenue',
 'inventory_assets',
 'inventory_revenue',
 'liabilities_assets',
 'liabilities_revenue',
 'payables_assets',
 'payables_revenue',
 'receivables_assets',
 'receivables_revenue',
 'tangibles_assets',
 'tangibles_revenue',
 'workingcapital_assets',
 'workingcapital_revenue']

In [26]:
data.dropna(subset=featureslist, inplace=True)
features = data.loc[:, featureslist].values

# Standardize features by removing the mean and scaling to unit variance.
# Fit to data, then transform it.
features = StandardScaler().fit_transform(features)

# GENERATE 4 PRINCIPAL COMPONENTS
pca = PCA(n_components = 4)
principal_components = pca.fit_transform(features)
principal_components

array([[ 1.04432342,  1.82526111, -0.83215424, -1.55303087],
       [ 0.87726126,  1.96347137, -0.5338174 , -1.84294994],
       [ 1.43815874,  1.7285305 , -0.08552984, -1.86433536],
       ...,
       [ 1.40509884,  0.84154668,  1.54804342, -0.29358349],
       [ 1.71709068, -1.08675286,  0.78496403, -1.63067361],
       [-3.1145692 , -2.16384076, -0.85000662, -1.35373966]])

In [27]:
pca.explained_variance_ratio_

array([0.20987584, 0.19637433, 0.11991552, 0.08598918])

In [28]:
pc_df = pd.DataFrame(principal_components)
pc_df

,0,1,2,3
0,1.044323,1.825261,-0.832154,-1.553031
1,0.877261,1.963471,-0.533817,-1.842950
2,1.438159,1.728531,-0.085530,-1.864335
3,1.049218,0.955964,-0.708829,-1.464093
4,1.754872,0.860990,-0.217008,-1.634615
...,...,...,...,...
31993,0.492243,0.052589,1.166264,-1.907583
31994,-2.956415,-1.026552,1.961500,3.848057
31995,1.405099,0.841547,1.548043,-0.293583
31996,1.717091,-1.086753,0.784964,-1.630674


### the principal components are orthogonal to each other

In [35]:
pc_df.corr()

,0,1,2,3
0,1.000000e+00,-4.312609e-16,-1.101986e-17,-2.447720e-17
1,-4.312609e-16,1.000000e+00,2.731008e-17,-1.939350e-16
2,-1.101986e-17,2.731008e-17,1.000000e+00,3.300276e-16
3,-2.447720e-17,-1.939350e-16,3.300276e-16,1.000000e+00


### Using raw variables, we note collinearity is a valid concern

In [36]:
data[featureslist].corr()

,cashneq_assets,cashneq_revenue,debt_assets,debt_revenue,ebit_assets,ebit_revenue,ebt_assets,ebt_revenue,eps_assets,eps_revenue,...,liabilities_assets,liabilities_revenue,payables_assets,payables_revenue,receivables_assets,receivables_revenue,tangibles_assets,tangibles_revenue,workingcapital_assets,workingcapital_revenue
cashneq_assets,1.000000,0.895256,-0.292677,-0.250374,0.175375,0.148728,0.207142,0.191860,0.079961,0.085604,...,-0.266556,-0.161553,-0.018300,-0.024520,0.014996,0.015357,0.195466,0.107419,0.435044,0.428894
cashneq_revenue,0.895256,1.000000,-0.192635,0.013885,0.049947,0.386767,0.079407,0.367008,-0.086386,0.083544,...,-0.275316,0.210214,-0.316719,-0.004697,-0.157946,0.173057,0.042534,0.475782,0.270564,0.523296
debt_assets,-0.292677,-0.192635,1.000000,0.918215,-0.094762,0.040022,-0.191457,-0.074686,-0.290557,-0.249276,...,0.684051,0.546871,-0.050293,0.079934,-0.119738,-0.004914,-0.251970,0.019848,-0.345498,-0.250882
debt_revenue,-0.250374,0.013885,0.918215,1.000000,-0.177426,0.262507,-0.259256,0.111811,-0.390816,-0.207106,...,0.552374,0.787336,-0.313327,0.083933,-0.256694,0.139164,-0.334821,0.358627,-0.402648,-0.089277
ebit_assets,0.175375,0.049947,-0.094762,-0.177426,1.000000,0.661322,0.946195,0.717450,0.396022,0.328675,...,-0.051216,-0.234414,0.054838,-0.154190,0.046100,-0.132743,0.130874,-0.172090,0.139640,-0.000694
ebit_revenue,0.148728,0.386767,0.040022,0.262507,0.661322,1.000000,0.617527,0.946308,0.061457,0.290370,...,-0.107046,0.416846,-0.474438,-0.100672,-0.258165,0.163868,-0.118842,0.512643,-0.087880,0.241132
ebt_assets,0.207142,0.079407,-0.191457,-0.259256,0.946195,0.617527,1.000000,0.768756,0.413775,0.348565,...,-0.154353,-0.295060,0.065485,-0.137602,0.075984,-0.100793,0.151357,-0.158529,0.181608,0.041990
ebt_revenue,0.191860,0.367008,-0.074686,0.111811,0.717450,0.946308,0.768756,1.000000,0.149476,0.332793,...,-0.197095,0.251320,-0.383249,-0.101845,-0.182938,0.142158,-0.055650,0.414320,-0.007597,0.244831
eps_assets,0.079961,-0.086386,-0.290557,-0.390816,0.396022,0.061457,0.413775,0.149476,1.000000,0.929428,...,-0.327984,-0.499110,0.090451,-0.214661,0.169785,-0.092951,0.220851,-0.238768,0.336295,0.127314
eps_revenue,0.085604,0.083544,-0.249276,-0.207106,0.328675,0.290370,0.348565,0.332793,0.929428,1.000000,...,-0.383242,-0.219438,-0.169295,-0.214265,0.029771,0.041710,0.118862,0.080923,0.254332,0.259590


In [37]:
pc_df.columns = ['PC1', 'PC2', 'PC3', 'PC4']
pc_df.index = data.index
pc_df

,PC1,PC2,PC3,PC4
datekey,,,,
2010-06-07,1.044323,1.825261,-0.832154,-1.553030
2010-09-07,0.877261,1.963471,-0.533817,-1.842950
2010-12-20,1.438159,1.728531,-0.085530,-1.864335
2011-03-09,1.049218,0.955964,-0.708829,-1.464093
2011-06-07,1.754872,0.860990,-0.217008,-1.634615
...,...,...,...,...
2020-10-08,0.492243,0.052589,1.166264,-1.907583
2020-10-08,-2.956415,-1.026552,1.961500,3.848058
2020-10-09,1.405099,0.841547,1.548043,-0.293583


In [38]:
data_merge = pd.concat([data, pc_df], axis=1)

# explain log(E/P ratio) using the raw variables
result = sm.OLS(data_merge['lnepratio'],
                sm.add_constant(data_merge[featureslist]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              lnepratio   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     189.2
Date:                Mon, 27 Sep 2021   Prob (F-statistic):               0.00
Time:                        19:02:15   Log-Likelihood:                -35919.
No. Observations:               31998   AIC:                         7.187e+04
Df Residuals:                   31982   BIC:                         7.200e+04
Df Model:                          15                                         
Covariance Type:              cluster                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 28, but rank is 15
  warnings.warn('covariance of constraints does not have full '


> Using raw variables:

>1. We know collinearity is a problem

>2. Interpreting the coefficients is tedious

In [39]:
# uses the PCs to explain log(E/P ratio)
result = sm.OLS(data_merge['lnepratio'],
                sm.add_constant(data_merge[['PC1', 'PC2', 'PC3', 'PC4']]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              lnepratio   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     86.46
Date:                Mon, 27 Sep 2021   Prob (F-statistic):           8.66e-72
Time:                        19:02:15   Log-Likelihood:                -39305.
No. Observations:               31998   AIC:                         7.862e+04
Df Residuals:                   31993   BIC:                         7.866e+04
Df Model:                           4                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2708      0.034   -125.770      0.0

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


> Using principal components:

>1. Collinearity is definitely not a problem

>2. But how do we interpret economic meaning of the coefficients?
For e.g., what does PC1 mean?

#### lets figure out what 'PC1' means

In [40]:
featureslist.append('PC1')
data_merge[featureslist].corr()

,cashneq_assets,cashneq_revenue,debt_assets,debt_revenue,ebit_assets,ebit_revenue,ebt_assets,ebt_revenue,eps_assets,eps_revenue,...,liabilities_revenue,payables_assets,payables_revenue,receivables_assets,receivables_revenue,tangibles_assets,tangibles_revenue,workingcapital_assets,workingcapital_revenue,PC1
cashneq_assets,1.000000,0.895256,-0.292677,-0.250374,0.175375,0.148728,0.207142,0.191860,0.079961,0.085604,...,-0.161553,-0.018300,-0.024520,0.014996,0.015357,0.195466,0.107419,0.435044,0.428894,0.505064
cashneq_revenue,0.895256,1.000000,-0.192635,0.013885,0.049947,0.386767,0.079407,0.367008,-0.086386,0.083544,...,0.210214,-0.316719,-0.004697,-0.157946,0.173057,0.042534,0.475782,0.270564,0.523296,0.591681
debt_assets,-0.292677,-0.192635,1.000000,0.918215,-0.094762,0.040022,-0.191457,-0.074686,-0.290557,-0.249276,...,0.546871,-0.050293,0.079934,-0.119738,-0.004914,-0.251970,0.019848,-0.345498,-0.250882,-0.474714
debt_revenue,-0.250374,0.013885,0.918215,1.000000,-0.177426,0.262507,-0.259256,0.111811,-0.390816,-0.207106,...,0.787336,-0.313327,0.083933,-0.256694,0.139164,-0.334821,0.358627,-0.402648,-0.089277,-0.280651
ebit_assets,0.175375,0.049947,-0.094762,-0.177426,1.000000,0.661322,0.946195,0.717450,0.396022,0.328675,...,-0.234414,0.054838,-0.154190,0.046100,-0.132743,0.130874,-0.172090,0.139640,-0.000694,0.514478
ebit_revenue,0.148728,0.386767,0.040022,0.262507,0.661322,1.000000,0.617527,0.946308,0.061457,0.290370,...,0.416846,-0.474438,-0.100672,-0.258165,0.163868,-0.118842,0.512643,-0.087880,0.241132,0.677813
ebt_assets,0.207142,0.079407,-0.191457,-0.259256,0.946195,0.617527,1.000000,0.768756,0.413775,0.348565,...,-0.295060,0.065485,-0.137602,0.075984,-0.100793,0.151357,-0.158529,0.181608,0.041990,0.572255
ebt_revenue,0.191860,0.367008,-0.074686,0.111811,0.717450,0.946308,0.768756,1.000000,0.149476,0.332793,...,0.251320,-0.383249,-0.101845,-0.182938,0.142158,-0.055650,0.414320,-0.007597,0.244831,0.735851
eps_assets,0.079961,-0.086386,-0.290557,-0.390816,0.396022,0.061457,0.413775,0.149476,1.000000,0.929428,...,-0.499110,0.090451,-0.214661,0.169785,-0.092951,0.220851,-0.238768,0.336295,0.127314,0.409809
eps_revenue,0.085604,0.083544,-0.249276,-0.207106,0.328675,0.290370,0.348565,0.332793,0.929428,1.000000,...,-0.219438,-0.169295,-0.214265,0.029771,0.041710,0.118862,0.080923,0.254332,0.259590,0.561851


>Looking just at raw variables with >40% correlation to PC1, we can hypothesize that it relates to “working capital / cash conversion cycle management” of the firm.

#### • The effect of an independent variable may depend on the value of another independent variable in the system

> • For instance, a variable on profitability may have a different predictive effect on stock price depending on whether the company’s revenue is growing or not

#### • Interaction variables allow us to model different slope coefficients in linear systems for different situations

#### • It is similar to dummy variables, where you can model different intercepts for various situations

## INTERACTION VARIABLES

> Generate the interaction variables. In this case, we create interaction variables as `lnoperatingmargin*industry dummy`

In [41]:
dummies = industrydummies.columns.values
dummies

array(['Agriculture Forestry And Fishing', 'Construction',
       'Finance Insurance And Real Estate', 'Manufacturing', 'Mining',
       'Retail Trade', 'Services',
       'Transportation Communications Electric Gas And Sanitary Service',
       'Wholesale Trade'], dtype=object)

In [42]:
dummies = dummies[1:len(dummies)-1]
dummies

array(['Construction', 'Finance Insurance And Real Estate',
       'Manufacturing', 'Mining', 'Retail Trade', 'Services',
       'Transportation Communications Electric Gas And Sanitary Service'],
      dtype=object)

In [43]:
dlist = []
for ind in dummies:
    tag = 'lnoperatingmargin'+'_'+ind
    data[tag] = data['lnoperatingmargin']*(data[ind])
    dlist.append(tag)
    dlist.append(ind)
dlist.append('lnoperatingmargin')
dlist

['lnoperatingmargin_Construction',
 'Construction',
 'lnoperatingmargin_Finance Insurance And Real Estate',
 'Finance Insurance And Real Estate',
 'lnoperatingmargin_Manufacturing',
 'Manufacturing',
 'lnoperatingmargin_Mining',
 'Mining',
 'lnoperatingmargin_Retail Trade',
 'Retail Trade',
 'lnoperatingmargin_Services',
 'Services',
 'lnoperatingmargin_Transportation Communications Electric Gas And Sanitary Service',
 'Transportation Communications Electric Gas And Sanitary Service',
 'lnoperatingmargin']

In [44]:
result = sm.OLS(data['lnepratio'],
                sm.add_constant(data[dlist]),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
result.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              lnepratio   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     22.55
Date:                Mon, 27 Sep 2021   Prob (F-statistic):           2.45e-61
Time:                        19:02:15   Log-Likelihood:                -41725.
No. Observations:               31998   AIC:                         8.348e+04
Df Residuals:                   31982   BIC:                         8.361e+04
Df Model:                          15                                         
Covariance Type:              cluster                                         
=====================================================================================================================================================
                                                                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                -3.6108      0.048    -75.842      0.000      -3.704      -3.518
lnoperatingmargin_Construction                                                        0.2473      0.109      2.266      0.023       0.033       0.461
Construction                                                                          0.7646      0.378      2.021      0.043       0.023       1.506
lnoperatingmargin_Finance Insurance And Real Estate                                  -0.1840      0.194     -0.947      0.343      -0.565       0.197
Finance Insurance And Real Estate                                                    -0.8341      0.417     -2.002      0.045      -1.651      -0.018
lnoperatingmargin_Manufacturing                                                      -0.0364      0.066     -0.555      0.579      -0.165       0.092
Manufacturing                                                                        -0.2274      0.106     -2.151      0.031      -0.435      -0.020
lnoperatingmargin_Mining                                                              0.3526      0.084      4.213      0.000       0.189       0.517
Mining                                                                                0.4767      0.143      3.334      0.001       0.196       0.757
lnoperatingmargin_Retail Trade                                                       -0.0927      0.082     -1.132      0.258      -0.253       0.068
Retail Trade                                                                         -0.2019      0.218     -0.926      0.354      -0.629       0.225
lnoperatingmargin_Services                                                            0.1619      0.060      2.720      0.007       0.045       0.279
Services                                                                             -0.0084      0.132     -0.063      0.950      -0.268       0.251
lnoperatingmargin_Transportation Communications Electric Gas And Sanitary Service     0.0424      0.075      0.567      0.571      -0.104       0.189
Transportation Communications Electric Gas And Sanitary Service                      -0.0560      0.163     -0.343      0.732      -0.376       0.264
lnoperatingmargin                                                                     0.2399      0.021     11.437      0.000       0.199       0.281
==============================================================================
Omnibus:                     8577.755   Durbin-Watson:                   1.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            89878.

> What does the coefficient of each interaction variable mean???

## SQUARED VARIABLES

### We explore the hypothesis that a stock’s earnings may have a concave relationship with its asset tangibility (plant property equipment / total assets)

In [45]:
data['tangibles_assets_normal'] = data['tangibles'] / data['assets']

data['tangibles_assets_sq'] = data['tangibles_assets_normal'] * data['tangibles_assets_normal']

result = sm.OLS(data['epratio'],
                sm.add_constant(data[['tangibles_assets_normal']]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                epratio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.04372
Date:                Mon, 27 Sep 2021   Prob (F-statistic):              0.834
Time:                        19:02:15   Log-Likelihood:            -1.0468e+05
No. Observations:               31998   AIC:                         2.094e+05
Df Residuals:                   31996   BIC:                         2.094e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


>  A univariate regression masks a more non-linear relationship.

In [46]:
result = sm.OLS(data['epratio'],
                sm.add_constant(data[['tangibles_assets_normal',
                                      'tangibles_assets_sq']]
                               ),
                missing='drop'
               ).fit(cov_type='cluster',
                     cov_kwds={'groups': data_w_dummies['siccode']}
                    )
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                epratio   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     1.743
Date:                Mon, 27 Sep 2021   Prob (F-statistic):              0.175
Time:                        19:02:15   Log-Likelihood:            -1.0468e+05
No. Observations:               31998   AIC:                         2.094e+05
Df Residuals:                   31995   BIC:                         2.094e+05
Df Model:                           2                                         
Covariance Type:              cluster                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


> Which we can uncover with a squared term!

## LAG INDEPENDENT VARIABLES

In [48]:
data_merge.index=[data_merge['ticker'], data_merge.index]

# lags the observations by 1 quarter
datalag1 = data_merge.groupby(level=0).shift(1)

dataset = pd.concat([data_merge['lnepratio'], datalag1[['PC1', 'PC2', 'PC3', 'PC4']]],
                    axis=1)
dataset.dropna(inplace=True)

result = sm.OLS(dataset['lnepratio'],
                sm.add_constant(dataset[['PC1', 'PC2', 'PC3', 'PC4']]
                               ),
                missing='drop'
               ).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              lnepratio   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     157.4
Date:                Mon, 27 Sep 2021   Prob (F-statistic):          1.57e-133
Time:                        19:08:13   Log-Likelihood:                -38303.
No. Observations:               29435   AIC:                         7.662e+04
Df Residuals:                   29430   BIC:                         7.666e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2796      0.005   -825.567      0.0

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [49]:
#F-TESTS
from statsmodels.formula.api import ols

formula = 'lnepratio ~ tangibles_assets_normal + tangibles_assets_sq + cashneq_assets + gp_assets + gp_revenue + fcf_assets'
result = ols(formula,
             data
            ).fit()

hypothesis1 = '(tangibles_assets_normal = cashneq_assets), tangibles_assets_sq = -0.5'
f_test = result.f_test(hypothesis1)

print(f_test)

<F test: F=array([[1633.20518947]]), p=0.0, df_denom=3.2e+04, df_num=2>


In [50]:
hypothesis2 = '(gp_assets = gp_revenue)'
f_Test2 = result.f_test(hypothesis2)

print(f_Test2)

<F test: F=array([[347.67837543]]), p=3.486258510969507e-77, df_denom=3.2e+04, df_num=1>


In [51]:
#deliberately get an insignificant F-test result so we can see what that looks like
formula = 'epratio ~ tangibles_assets_normal + tangibles_assets_sq + cashneq_assets + gp_assets + gp_revenue + fcf_assets'
result = ols(formula,
             data
            ).fit()

f_Test2 = result.f_test(hypothesis2)
print(f_Test2)

<F test: F=array([[0.09223886]]), p=0.761351945678713, df_denom=3.2e+04, df_num=1>


## HAZARD models

In [52]:
survival_df = pd.read_csv("survivaldata.csv")
# drop data where delisted status is unsure
survival_df.dropna(subset=['isdelisted'], inplace=True)

# code a 1/0 variable based on whether delisted or not.  Python needs a numerical value
survival_df.loc[survival_df.isdelisted=='N', 'dead'] = 0
survival_df.loc[survival_df.isdelisted=='Y', 'dead'] = 1

# estimate probabiltiy of delisting based on revenue category
sizedummies = pd.get_dummies(survival_df['scalerevenue'])
survival_df = pd.concat([survival_df, sizedummies], axis=1)

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
import statsmodels.formula.api as smf

# purely for convenience
survival_df.rename(columns = {'1 - Nano':'Nano',
                              '2 - Micro': 'Micro',
                              '3 - Small': 'Small',
                              '4 - Mid': 'Mid',
                              '5 - Large': 'Large',
                              '6 - Mega': 'Mega'},
                   inplace=True)

mod = smf.phreg("duration ~ 0 + Nano + Micro + Small + Mid + Large + Mega",
                survival_df,
                status = survival_df["dead"].values,
                ties = "efron"
               )
rslt = mod.fit()
rslt.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/duration/hazard_regression.py:425: UserWarning: PHReg formulas should not include any '0' or '1' terms
  warnings.warn("PHReg formulas should not include any '0' or '1' terms")


<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: PHReg
============================================================
Model:                   PH Reg       Sample size:     37850
Dependent variable:      duration     Num. events:     22114
Ties:                    Efron                              
------------------------------------------------------------
       log HR log HR SE   HR      t     P>|t|  [0.025 0.975]
------------------------------------------------------------
Nano   0.2059    0.0219 1.2286   9.4155 0.0000 1.1770 1.2824
Micro  0.0195    0.0211 1.0197   0.9236 0.3557 0.9784 1.0627
Small -0.3306    0.0222 0.7185 -14.8766 0.0000 0.6879 0.7505
Mid   -0.9237    0.0306 0.3970 -30.2113 0.0000 0.3739 0.4216
Large -1.7264    0.0517 0.1779 -33.3990 0.0000 0.1608 0.1969
Mega  -2.9883    0.5004 0.0504  -5.9725 0.0000 0.0189 0.1343
============================================================
Confidence intervals are for the hazard ratios
"""

In [54]:
survival_df['sizecat'] = 0
survival_df.loc[survival_df['Micro']==1, 'sizecat'] = 1
survival_df.loc[survival_df['Small']==1, 'sizecat'] = 2
survival_df.loc[survival_df['Mid']==1, 'sizecat'] = 3
survival_df.loc[survival_df['Large']==1, 'sizecat'] = 4
survival_df.loc[survival_df['Mega']==1, 'sizecat'] = 5
survival_df.sizecat

0        4
1        4
2        0
3        3
4        1
        ..
37845    0
37846    0
37847    0
37848    0
37849    0
Name: sizecat, Length: 37850, dtype: int64

In [55]:
mod = smf.phreg("duration ~ 0 + sizecat",
                survival_df,
                status = survival_df["dead"].values,
                ties = "efron"
               )
rslt = mod.fit()
rslt.summary()

/Users/xiezijun/Documents/anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/duration/hazard_regression.py:425: UserWarning: PHReg formulas should not include any '0' or '1' terms
  warnings.warn("PHReg formulas should not include any '0' or '1' terms")


<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: PHReg
==============================================================
Model:                   PH Reg        Sample size:      37850
Dependent variable:      duration      Num. events:      22114
Ties:                    Efron                                
--------------------------------------------------------------
         log HR log HR SE   HR      t     P>|t|  [0.025 0.975]
--------------------------------------------------------------
sizecat -0.3376    0.0061 0.7135 -55.3201 0.0000 0.7050 0.7220
==============================================================
Confidence intervals are for the hazard ratios
"""